In [ ]:
!pip install qdrant-client sentence-transformers transformers torch accelerate openai bitsandbytes

In [49]:
import json, uuid, os, torch, openai, re 
from typing import List, Dict, Any
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient
from qdrant_client.models import PointStruct, VectorParams, Distance
from google.colab import userdata

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {device}")

Device: cuda


### DB 구축하기

In [ ]:
embedding_model = SentenceTransformer('dragonkue/BGE-m3-ko')

In [4]:
DB_PATH = "/content/drive/MyDrive/yonsei/YAI/qdrant_bge"
COLLECTION_NAME = "dcinside"

client = QdrantClient(path=DB_PATH)

collections = client.get_collections()
collection_names = [c.name for c in collections.collections]

if COLLECTION_NAME in collection_names:
    print(f" existing collection '{COLLECTION_NAME}'")
else:
    client.create_collection(
        collection_name=COLLECTION_NAME,
        vectors_config=VectorParams(size=1024, distance=Distance.COSINE)
    )
    print(f" new collection '{COLLECTION_NAME}' ")

 existing collection 'dcinside'


인덱스 생성하기

In [ ]:
data_path = "/content/drive/MyDrive/yonsei/YAI/merged_dataset.json"

with open(data_path, 'r', encoding='utf-8') as f:
    dataset = json.load(f)

points_to_upsert = []
batch_size = 100

for idx, item in enumerate(dataset):
    main_text = item.get('main', '')
    comments = item.get('comments', [])
    comments_text = ' '.join(comments) if comments else ''

    full_text = f"{main_text} {comments_text}".strip()

    if not full_text:
        continue

    vector = embedding_model.encode(full_text).tolist()

    payload = {
        "date": item.get('date', ''),
        "main": main_text,
        "comments": comments,
        "source_url": item.get('source_url', ''),
        "gallery": item.get('gallery', ''),
        "full_text": full_text
    }

    point = PointStruct(
        id=str(uuid.uuid4()),
        vector=vector,
        payload=payload
    )
    points_to_upsert.append(point)

    if len(points_to_upsert) >= batch_size:
        client.upsert(
            collection_name=COLLECTION_NAME,
            points=points_to_upsert,
            wait=True
        )
        print(f"Indexed {idx + 1}/{len(dataset)} items...")
        points_to_upsert = []

if points_to_upsert:
    client.upsert(
        collection_name=COLLECTION_NAME,
        points=points_to_upsert,
        wait=True
    )

print(f"index completed : {len(dataset)}")

In [ ]:
openai.api_key = userdata.get('OPENAI_KEY')

### RAG 함수 정의

In [5]:
def search_documents(query: str, top_k: int = 3, gallery_filter: str = None) -> List[Dict]:
    query_vector = embedding_model.encode(query).tolist()

    query_filter = None
    if gallery_filter:
        from qdrant_client.models import Filter, FieldCondition, MatchValue
        query_filter = Filter(
            must=[
                FieldCondition(
                    key="gallery",
                    match=MatchValue(value=gallery_filter)
                )
            ]
        )

    search_response = client.query_points(
        collection_name=COLLECTION_NAME,
        query=query_vector,
        query_filter=query_filter,
        limit=top_k,
        with_payload=True
    )

    results = []
    for hit in search_response.points:
        results.append({
            "score": hit.score,
            "date": hit.payload.get('date', ''),
            "main": hit.payload.get('main', ''),
            "comments": hit.payload.get('comments', []),
            "source_url": hit.payload.get('source_url', ''),
            "gallery": hit.payload.get('gallery', ''),
            "full_text": hit.payload.get('full_text', '')
        })

    return results

In [6]:
def _prepare_rag_context(query: str, top_k: int = 3, gallery_filter: str = None) -> tuple:
    retrieved_docs = search_documents(query, top_k=top_k, gallery_filter=gallery_filter)

    context_parts = []
    for i, doc in enumerate(retrieved_docs, 1):
        context_parts.append(f"[문서 {i}]")
        context_parts.append(f"날짜: {doc['date']}")
        context_parts.append(f"갤러리: {doc['gallery']}")
        context_parts.append(f"내용: {doc['main']}")
        if doc['comments']:
            context_parts.append(f"댓글: {', '.join(doc['comments'][:3])}")
        context_parts.append("")

    context = "\n".join(context_parts)
    return retrieved_docs, context

api로 데려오기

In [ ]:
def generate_rag_response_api(
    query: str,
    model: str = "gpt-4o-mini",
    top_k: int = 3,
    gallery_filter: str = None,
    max_tokens: int = 512,
    temperature: float = 0.7
) -> Dict[str, Any]:

    retrieved_docs, context = _prepare_rag_context(query, top_k, gallery_filter)
    prompt = f"""다음 문서에 있는대로 질문에 답변해주세요.

{context}

질문: {query}

답변:"""

    response = openai.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "당신은 제공된 문서에 있는대로 답변하는 도우미입니다."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=max_tokens,
        temperature=temperature
    )
    answer = response.choices[0].message.content.strip()

    return {
        "query": query,
        "answer": answer,
        "retrieved_docs": retrieved_docs,
        "model_used": model
    }

In [ ]:
!ls -lh ~/.cache/huggingface/hub | grep models

drwxr-xr-x 6 root root 4.0K Nov 21 05:35 models--dragonkue--BGE-m3-ko
drwxr-xr-x 6 root root 4.0K Nov 21 07:47 models--naver-hyperclovax--HyperCLOVAX-SEED-Text-Instruct-1.5B
drwxr-xr-x 6 root root 4.0K Nov 21 06:20 models--QuixiAI--WizardLM-7B-Uncensored


로컬 모델로 데려오기

In [ ]:
# model_name = "naver-hyperclovax/HyperCLOVAX-SEED-Text-Instruct-1.5B"
# model_name = "EleutherAI/polyglot-ko-5.8b"
# model_name = "LGAI-EXAONE/EXAONE-3.0-7.8B-Instruct"
# model_name = "QuixiAI/WizardLM-7B-Uncensored" 한국어를 못함
# model_name = "MLP-KTLim/llama-3-Korean-Bllossom-8B" 코랩에서 안돌아감 (양자화 필요)
# model_name = "Qwen/Qwen2.5-3B"
# model_name = "chuanli11/Llama-3.2-3B-Instruct-uncensored"
# model_name = "bartowski/Llama-3.2-3B-Instruct-uncensored-GGUF"
model_name = "s5ya/Ko-Llama-3.1-8B-Lexi-Uncensored-V2"

# 양자화 필요할 시 (코랩 기준 8B 이상)
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quantization_config, # 필요할 시
    torch_dtype=torch.float16 if device == "cuda" else torch.float32,
    device_map="auto" if device == "cuda" else None,
    low_cpu_mem_usage=True
)

print('load done !')

In [44]:
def generate_rag_response_local(
    query: str,
    top_k: int = 10, # [수정] 5 -> 10 (더 많이 긁어옴)
    gallery_filter: str = None,
    max_tokens: int = 150, 
    temperature: float = 0.45 
) -> Dict[str, Any]:

    # 1. 문서 검색
    retrieved_docs, _ = _prepare_rag_context(query, top_k, gallery_filter)

    # [디버깅] 검색된 원본 개수 출력
    print(f"\n🔍 검색된 원본 문서 개수: {len(retrieved_docs)}개")

    # 2. 중복 제거 및 내용 병합
    unique_docs = []
    seen_contents = set()
    
    for doc in retrieved_docs:
        # 본문과 댓글을 모두 합쳐서 풍부한 Context 만들기
        content_parts = [doc['main'].strip()]
        if doc['comments']:
            # 댓글도 내용에 포함 (너무 짧은 본문 보완)
            content_parts.extend(doc['comments'][:2])
        
        full_content = " ".join(content_parts)
        
        # 내용이 너무 짧거나 중복이면 패스, 아니면 추가
        if len(full_content) > 2 and full_content not in seen_contents:
            unique_docs.append(doc)
            seen_contents.add(full_content)

    # [디버깅] 중복 제거 후 개수 출력
    print(f"✨ 중복 제거 후 남은 문서: {len(unique_docs)}개")
    
    # Context 구성
    context_list = []
    for doc in unique_docs:
        # 본문 + 댓글 결합
        text = doc['main']
        if doc['comments']:
            text += " (" + " ".join(doc['comments'][:2]) + ")"
        context_list.append(text)
        
    context = "\n".join(context_list)

    print("\n" + "="*20 + " [Context Preview] " + "="*20)
    print(context[:500] + "..." if len(context) > 500 else context)
    print("="*60 + "\n")

    # 문서가 없으면 바로 리턴
    if not context.strip():
        return {
            "query": query,
            "answer": "관련된 내용이 DB에 없음.",
            "retrieved_docs": []
        }

    # 3. 프롬프트 구성 (부정 명령 제거 -> 양성 명령 전환)
    messages = [
        {
            "role": "system",
            "content": (
                "너는 디시인사이드 갤러리 유저다. "
                "주어진 [텍스트]들의 내용을 종합해서 **반말(비속어, 음슴체)**로 한 문장으로 요약해라. "
                "니 생각은 넣지 말고, 텍스트에 있는 여론만 그대로 읊어라."
            )
        },
        {
            "role": "user",
            "content": (
                f"아래 [텍스트] 읽고 디시 말투로 한 줄 요약해.\n"
                f"내용:\n{context}\n\n"
                f"요약:"
            )
        }
    ]

    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(device)

    terminators = [
        tokenizer.eos_token_id,
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    # 4. 생성
    with torch.no_grad():
        outputs = model.generate(
            input_ids,
            max_new_tokens=max_tokens,
            eos_token_id=terminators,
            do_sample=True,
            temperature=temperature,
            top_p=0.9,
            repetition_penalty=1.1, # 1.1로 안전하게 복귀
            no_repeat_ngram_size=0,
        )

    # 5. 답변 추출
    response = outputs[0][input_ids.shape[-1]:]
    answer = tokenizer.decode(response, skip_special_tokens=True).strip()

    # [후처리]
    if "요약:" in answer:
        answer = answer.split("요약:")[-1].strip()
    
    # 불필요한 기호/영어 제거 (한글, 숫자, 문장부호만)
    answer = re.sub(r'[^가-힣0-9\s.,!?~ㅋㅎ]', '', answer)
    answer = re.sub(r'\s+', ' ', answer).strip()

    return {
        "query": query,
        "answer": answer,
        "retrieved_docs": unique_docs
    }

테스트 1. 질문에 답해보거라

In [45]:
result = generate_rag_response_local("여자도 군대에 가야한다고 생각해?")
print(result)

{'query': '여자도 군대에 가야한다고 생각해?', 'answer': 'Answer the question based ONLY on the provided documents. \n  Do not make up information.\n\n  Documents:\n  [문서 1]\n날짜: 2024.05.12 13:19:15\n갤러리: \n내용: 여성은 군대 3년 보내라\n댓글: 이, ㅇㅈ\n\n[문서 2]\n날짜: 2025.03.12 23:47:04\n갤러리: \n내용: 나 여잔데 군대가고싶음\n댓글: 통일되도 여자도 군대가는 제도로 바꿔야함\n\n[문서 3]\n날짜: 2023.05.12 17:59:50\n갤러리: \n내용: 여자가 군대가는건 애 안낳아서 가는거.\n댓글: 천재인데?\n\n\n  Question: 여자도 군대에 가야한다고 생각해?\n\n  Answer in Korean based on the documents: \n\n   (no answer)    // if there is no relevant document or clear opinion expressed\n\n\n      (yes/strongly agree)\n        : \n\n\n       (somehow/agree)\n         : \n\n\n\n     (disagree/somewhat disagree)\n          : \n\n\n\n\n     (clearly/dismissively disagree)\n           : \n\n\n\n\nNote:\n\n* The response should be indicated by one of the above options only.\n* If a strong emotion like "hate" or "love" appears in any comment but it\'s related to another user and not directly answering your question then ignore t

In [ ]:
def interactive_rag():
    while True:
        user_input = input("\n질문: ").strip()
        if user_input.lower() in ['quit', 'exit']:
            print("\n대화를 종료합니다.")
            break

In [ ]:
interactive_rag()